In [59]:
import pandas as pd
import numpy as np
agora_data = pd.read_json('agora_hack_products.json')
agora_data

,product_id,name,props,is_reference,reference_id
0,0007302f2fe1d54d,Классическая сплит-система ROYAL CLIMA PANDORA...,"[Класс энергоэффективности\tA, Мощность конд...",False,f497219eb0077f84
1,000740b6c1cc763e,Смартфон Xiaomi Redmi Note 10S NFC 6/128 ГБ RU...,"[Экран\t6.43"" (2400x1080) AMOLED 60 Гц, 4 каме...",True,None
2,0039af5efceac4ab,Холодильник Бирюса 118,[Мощность замораживания 4 кг/сутки],False,28085e941cde1639
3,004f2158acb8165c,ASUS TUF-GTX1660S-O6G-GAMING Видеокарта,"[Объем видеопамяти\t6144 МБ, Частота памяти\...",False,9afe55bb4bf1e8a8
4,005cddb29e1677ec,"Кофемашина Saeco Lirika One Touch Cappuccino, ...","[Приготовление капучино\tавтоматическое, Мате...",False,1f21918ceb5d345c
...,...,...,...,...,...
3246,ff5da4be6fa60c4b,"15.6"" Ноутбук Lenovo IdeaPad Gaming 315IHU6 19...","[Видеокарта\tNVIDIA GeForce RTX 3050 4 ГБ, Вре...",True,None
3247,ff66532467a02652,Моющий робот-пылесос для дома и квартиры Xiaom...,"[Объем контейнера для воды\t0.27 л, ШхГхВ\t35....",False,d4ebc4a26700d5e0
3248,ff75ade409f4da7e,"Huawei Умный браслет Band 7, графитово-черный","[Модификация\tGPS, Материал корпуса\tпластик]",False,177ccb3b84125efa
3249,ffb770de0c2feafc,Видеокарта Gigabyte GV-R675XTGAMING OC-12GD 12...,[Комплектация\tRetail],False,a07d5538ebec8e36


## деление датасета на эталоны и товары

In [60]:
agora_data_goods = agora_data[agora_data['is_reference'] == False]
agora_data_prime = agora_data[agora_data['is_reference'] == True]

## токенизация имён

In [61]:
import re
reg = re.compile('[^a-zа-я0-9 ]')

Xname = {}
Xprops = []
for index, row in agora_data_goods.iterrows():
    tmp = row['name'].lower()
    tmp = reg.sub('', tmp).replace("ё", "е").split(" ")
    Xname[agora_data_goods.product_id[index]] = set(tmp)
    tmp = ' '.join(row['props']).replace("\t", " ").lower()
    tmp = reg.sub('', tmp).replace("ё", "е").split(" ")
    Xprops.append(set(tmp))

etalonsname = {}
etalonsprops = []
for index, row in agora_data_prime.iterrows():
    tmp = row['name'].lower()
    tmp = reg.sub('', tmp).replace("ё", "е").split(" ")
    etalonsname[agora_data_prime.product_id[index]] = set(tmp)
    tmp = ' '.join(row['props']).replace("\t", " ").lower()
    tmp = reg.sub('', tmp).replace("ё", "е").split(" ")
    etalonsprops.append(set(tmp))

y = np.array(agora_data_goods.reference_id)
etalonsname

{'000740b6c1cc763e': {'10s',
  '6128',
  'nfc',
  'note',
  'redmi',
  'ru',
  'xiaomi',
  'белоснежная',
  'галька',
  'гб',
  'смартфон'},
 '0083737f904dd9a9': {'lu1058', 'lumme', 'фен'},
 '01378fa92f901df5': {'156',
  '15are05',
  '1920x1080',
  '23',
  '256',
  '3',
  '4500u',
  '5',
  '81w40033rk',
  'amd',
  'dos',
  'graphics',
  'grey',
  'ideapad',
  'lenovo',
  'platinum',
  'radeon',
  'ryzen',
  'ssd',
  'гб',
  'ггц',
  'ноутбук'},
 '01601abb0bf5133d': {'hymd2071',
  'hyundai',
  'микроволновая',
  'печь',
  'серебристый'},
 '01a28e17323d6ef8': {'3020bi', 'phaser', 'xerox', 'лазерный', 'принтер'},
 '01e52344308143f6': {'g4g02',
  'weissgauff',
  'wgs',
  'ws',
  'белый',
  'газовая',
  'плита'},
 '0229abcf2ded1058': {'525', 'caferomatica', 'nicr', 'nivona', 'кофемашина'},
 '05a95e7066dc4c52': {'7',
  'band',
  'global',
  'mi',
  'xiaomi',
  'браслет',
  'умный',
  'черный'},
 '05cb692a7fd6c0e6': {'33', 'defender', 'spk', 'акустика', 'компьютерная'},
 '05d4f6d7e4fe8c4b': {

Сравниваем токены у товаров и эталонов

In [62]:
weight1 = 1
weight2 = 0
ans = []
for i, j in zip(Xname.values(), Xprops):
    max_len = 0
    tmp_ans = 0
    for k, l, m in zip(etalonsname.keys(), etalonsname.values(), etalonsprops):
        lendiff = abs(len(i) - len(l))
        if (weight1 * len(i & l) + len(m & j) - weight2 * lendiff > max_len):
            max_len = len(i & l) + len(m & j)
            tmp_ans = k
    ans.append(tmp_ans)

In [63]:
from sklearn.metrics import accuracy_score
accuracy_score(y, np.array(ans))

0.960431654676259

In [56]:
df = pd.DataFrame({'true': y, 'pred': np.array(ans), 'prod_name': np.array(Xname.values()), 'prod_props': np.array(Xprops)})
df_wrong = df.where(df.true != df.pred)

In [57]:
df_wrong.dropna().to_csv("1.csv")